# Volatility-timing Strategy

## Summary

Mean-variance portfolio theory tells us that an investor chooses the tangency portfolio, i.e. portfolio of risky assets with the highest Sharpe ratio:

$$
SR \equiv \frac{\mathbb{E}\left[r\right]-r_f}{\sigma_{r-r_f}},
$$

and combine the tangency portfolo with the risk-free asset to form an optimal portfolio. The weight on the tangency portfolio is given by

$$
w = \frac{SR}{RA \times \sigma_{r-r_f}},
$$

where RA is risk aversion.

In the CAPM, all investors hold a mean-variance efficient portfolio and as supply has to equal demand, that is, $w = 1$. Consequently, it holds

$$
\mathbb{E}\left[r\right]-r_f = RA \times \sigma_{r-r_f}.
$$

Hence, the risk premium depends on risk aversion and the market's volatility. In reality, if the risk premium does not move enough to compensate for variation in volatility, there is scope to improve the risk-return trade-off of our strategy by volatility timing.

<img src="../Figures/trial.png" alt="Drawing" style="width: 600px;"/>

## Reference

[Moreira and Muir, Volatility-Managed Portfolios, Journal of Finance, 2017.](https://onlinelibrary.wiley.com/doi/full/10.1111/jofi.12513)

## Quantopian Implementation

### We follow the five steps in building an investment strategy:

1. What is the economic idea we want to test?
A: Can we generate alpha by reducing our allocation to the market when the market volatility is high.

2. What is the universe of securities we want to trade?
A: The market portfolio, which we approximate by an ETF on the S&P500 (ticker: SPY).

3. What is the signal on which we want to trade?
A: Recent volatility, say, over the last 20 trading days

4. How do we construct our portfolio?
A: Weight in the market at time t = 0.2 / Annualized volatility at time t.

5. What is the benchmark? 
A: The ETF itself.

### Now we can build our strategies in Quantopian in four steps:

Initialize the strategy:

This part of the code will be computed once only.

In [ ]:
def initialize(context):
    
    context.SPY = symbol('SPY')
    
    schedule_function(rebalance, 
                      date_rules.month_start(),
                      time_rules.market_open())
    
    schedule_function(record_vars, 
                     date_rules.every_day(),
                     time_rules.market_close())

Compute weights:

This part of the code will be computed every time (for instance, every day, week or month) when we trade and computes the weights we want to hold.

In [ ]:
def compute_weights(context, data):
    
    prices = data.history(context.SPY, 'price', 20, '1d')
    returns = prices.pct_change()
    std_20 = returns.std()*np.sqrt(250)
    
    weight = min(0.2 / std_20, 1.5)
    
    return weight

Rebalance:

This part of the code will be computed every time we trade and executes the actual transactions.

In [ ]:
def rebalance(context, data):
    
    weight = compute_weights(context, data)
    
    if data.can_trade(context.SPY):
        order_target_percent(context.SPY,weight)

Record variables:

Here we can record key properties of our strategy. How many long and short positions do we have? How much leverage do we have? We can also choose how often to record the variables, which may be different from how frequently we trade.

In [ ]:
def record_vars(context, data):
    record(leverage = context.account.leverage, cash = context.portfolio.cash/context.portfolio.portfolio_value)

### Here is the complete code:

In [ ]:
import numpy as np

def initialize(context):
    
    context.SPY = symbol('SPY')
    
    schedule_function(rebalance, 
                      date_rules.month_start(),
                      time_rules.market_open())
    
    schedule_function(record_vars, 
                     date_rules.every_day(),
                     time_rules.market_close())
    
def record_vars(context, data):
    record(leverage = context.account.leverage, cash = context.portfolio.cash/context.portfolio.portfolio_value)
    
def compute_weights(context, data):
    prices = data.history(context.SPY, 'price', 20, '1d')
    returns = prices.pct_change()
    std_20 = returns.std()*np.sqrt(250)
    
    weight = min(0.2 / std_20, 1.5)
    
    return weight
    
def rebalance(context, data):
    
    weight = compute_weights(context, data)
    
    if data.can_trade(context.SPY):
        order_target_percent(context.SPY,weight)